# import

In [ ]:
import numpy as np
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm
import torch
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from kiwipiepy import Kiwi

from sklearn.decomposition import LatentDirichletAllocation
import gensim
from gensim.models.coherencemodel import CoherenceModel
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel
from bertopic import BERTopic
from sklearn.pipeline import make_pipeline

import gensim.corpora as corpora
from sklearn.decomposition import NMF
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from gensim.corpora import Dictionary

from mpl_toolkits.mplot3d import Axes3D
from sklearn.feature_extraction.text import CountVectorizer
from hdbscan import HDBSCAN


from sklearn.pipeline import make_pipeline
from sklearn.decomposition import TruncatedSVD

from bertopic.backend import BaseEmbedder
from sklearn.utils.validation import check_is_fitted, NotFittedError
from gensim.models import Word2Vec
from gensim.models import KeyedVectors


- 토픽의 개수는 각 구마다 평균 부서 숫자가 35이기 때문에 35로 설정함
- 토픽모델링에서 일반적인 성능지표로 쓰이는 coherence_score 를 사용하여 성능을 비교하였다

# 전처리

In [ ]:
gwa_df = pd.read_csv('../data/0. 구_과_하는일_통합.csv')
minwon_df = pd.read_csv('../data/0. 새올_통합.csv')

In [ ]:
kiwi = Kiwi()

def extract_text(text, exclude_keywords=None):
    result = kiwi.tokenize(text)
    excluded_tokens = set(exclude_keywords) if exclude_keywords else set()
    tokens = [token.form for token in result if token.tag in ['NNG', 'NNP'] and token.form not in excluded_tokens and len(token.form) > 1]
    return ' '.join(tokens)


def extract_token(text, exclude_keywords=None):
    result = kiwi.tokenize(text)
    excluded_tokens = set(exclude_keywords) if exclude_keywords else set()
    tokens = [token.form for token in result if token.tag in ['NNG', 'NNP'] and token.form not in excluded_tokens and len(token.form) > 1]
    return tokens


In [ ]:
# 논의를 통한 불용어 사전 정의
li = ['관리','업무','운영','사업','지원','총괄','관련','계획','시설','민원','센터','처리','추진','유지','조사','수립','예산','신고','지역','사항','시스템','정보','위원회','통합','차량','지도','공공','자료','평가','접수','담당','종합','개선','현장','직원','변경','구역','기관','서울','사례','회계','주요','정책','기획','협의','활성']

In [ ]:
# 불용어 사전을 적용하지 않은 토큰화된 칼럼과 텍스트 칼럼 생성

gwa_df['token'] =  gwa_df['하는일'].apply(extract_token)
gwa_df['text'] =  gwa_df['하는일'].apply(extract_text)


In [ ]:
# 불용어 사전을 적용한 토큰화된 칼럼과 텍스트 칼럼 생성

gwa_df['ex_token'] = gwa_df['하는일'].apply(lambda text: extract_token(text, exclude_keywords=li))
gwa_df['ex_text'] = gwa_df['하는일'].apply(lambda text: extract_text(text, exclude_keywords=li))


# LDA

In [ ]:


def tfvec(texts, num):
    # 1. LDA 모델링
    dictionary = Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]
    lda_model = LdaModel(corpus, num_topics=num, id2word=dictionary, random_state=4)

    # 2. 각 문서에 대해 가장 높은 확률을 가진 토픽 선택
    lda_topic = [max(lda_model.get_document_topics(item), key=lambda x: x[1])[0] for item in corpus]

    # 3. 선택된 토픽과 그에 해당하는 단어들을 데이터 프레임에 저장
    topics_list = []
    topics_words_list = [] # CoherenceModel에 사용할 토픽의 단어 리스트
    for topic_id, topic in lda_model.show_topics(num_topics=num, num_words=20, formatted=False):
        topic_words = [word[0] for word in topic]
        topics_list.append({'Topic': topic_id, 'Words': " ".join(topic_words)})
        topics_words_list.append(topic_words) # 각 토픽의 단어들을 리스트에 추가

    topics_df = pd.DataFrame(topics_list)

    # 4. coherence_score 구하기
    coherence_model = CoherenceModel(topics=topics_words_list, texts=texts, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()

    return lda_topic, topics_df, coherence_score


In [ ]:
# 함수 호출
topics, topics_df, score = tfvec(gwa_df['token'], num=35)


In [ ]:
score

0.30308447864604476

In [ ]:
topics, topics_df, score = tfvec(gwa_df['ex_token'], num=35)
print(score)

0.3242473293463132


# NMF

In [ ]:


def nmf_topic_modeling(texts, num):
    # 1. TF-IDF 벡터화
    tfidf_vectorizer = TfidfVectorizer(max_df=0.98, min_df=2)
    tfidf = tfidf_vectorizer.fit_transform([" ".join(text) for text in texts])

    # 2. NMF 모델링
    nmf_model = NMF(n_components=num, random_state=4, l1_ratio=.5, init='nndsvd').fit(tfidf)

    # 3. 각 문서에 대해 가장 높은 확률을 가진 토픽 선택
    nmf_topics = nmf_model.transform(tfidf)
    most_prob_topic = nmf_topics.argmax(axis=1)

    # 4. 선택된 토픽과 그에 해당하는 단어들을 데이터 프레임에 저장
    topics_list = []
    topics_words_list = []  # CoherenceModel에 사용할 토픽의 단어 리스트
    feature_names = tfidf_vectorizer.get_feature_names_out()
    for topic_idx, topic in enumerate(nmf_model.components_):
        topic_words = [feature_names[i] for i in topic.argsort()[:-20-1:-1]]  # 상위 20개 단어
        topics_list.append({'Topic': topic_idx, 'Words': " ".join(topic_words)})
        topics_words_list.append(topic_words) # 각 토픽의 단어들을 리스트에 추가

    topics_df = pd.DataFrame(topics_list)

    # 5. Coherence score 계산
    dictionary = Dictionary(texts)
    coherence_model = CoherenceModel(topics=topics_words_list, texts=texts, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()

    return most_prob_topic, topics_df, coherence_score



In [ ]:
topics, topics_df, coherence = nmf_topic_modeling(gwa_df['token'].tolist(), num=35)

In [ ]:
coherence

0.6960640193939929

In [ ]:
topics, topics_df, coherence = nmf_topic_modeling(gwa_df['ex_token'].tolist(), num=35)

In [ ]:
print(coherence)

0.8290314129732812


# BERTopic

In [ ]:
# 점수를 계산하기 위한 함수
def score(df, texts):
    # Gensim dictionary 및 corpus 생성
    dictionary = Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]
    # Representation 컬럼으로부터 토픽 리스트 생성
    topic = df['Representation']
    # Coherence 계산
    coherence_model = CoherenceModel(topics=topic, texts=texts, dictionary=dictionary, coherence='c_v')
    coherence_value = coherence_model.get_coherence()

    return coherence_value

In [ ]:
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained("klue/bert-base")
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

In [ ]:
vectorizer = CountVectorizer(tokenizer=extract_token, min_df=2)
hdbscan_model = HDBSCAN(min_cluster_size=20, metric='euclidean', cluster_selection_method='leaf', prediction_data=False)

In [ ]:
# bertopic을 하기 위한 데이터프레임
def topicmodel(model, vectorizer, nr_topics, top_n_words, texts, embeddings=None):

    topic_model = BERTopic(embedding_model=model,
                       vectorizer_model=vectorizer,
                       nr_topics=nr_topics,
                       top_n_words=top_n_words,
                       calculate_probabilities=False)

    topics, probs = topic_model.fit_transform(texts.tolist(), embeddings=embeddings)
    first_result = topic_model.get_topic_info()

    return topics, first_result


## word2vec

### 불용어 사전 적용 x

In [ ]:
sentences = gwa_df['text']
model_w2v = Word2Vec(sentences, vector_size=200, window=5, min_count=1, workers=4)


# Word2VecKeyedVectors 인스턴스
word_vectors = model_w2v.wv
# 문서 임베딩 계산: 문서 내의 모든 단어의 임베딩을 평균냄
def document_vector(doc, model):
    # 해당 문서에 있는 단어 중 모델에 학습된 단어에 대한 임베딩만 사용
    doc = [word for word in doc if word in model.wv]
    if len(doc) == 0:
        return np.zeros(model.vector_size)
    else:
        return np.mean(model.wv[doc], axis=0)

doc_embeddings = np.array([document_vector(doc, model_w2v) for doc in sentences])



In [ ]:
topics, first_result = topicmodel(word_vectors, vectorizer, 35, 20, gwa_df['text'], doc_embeddings)

In [ ]:
score(first_result, gwa_df['token'])

0.6805869237420512

### 불용어 사전 o

In [ ]:
sentences = gwa_df['ex_text']
model_w2v = Word2Vec(sentences, vector_size=200, window=5, min_count=1, workers=4)


# Word2VecKeyedVectors 인스턴스
word_vectors = model_w2v.wv
# 문서 임베딩 계산: 문서 내의 모든 단어의 임베딩을 평균냄
def document_vector(doc, model):
    # 해당 문서에 있는 단어 중 모델에 학습된 단어에 대한 임베딩만 사용
    doc = [word for word in doc if word in model.wv]
    if len(doc) == 0:
        return np.zeros(model.vector_size)
    else:
        return np.mean(model.wv[doc], axis=0)

doc_embeddings = np.array([document_vector(doc, model_w2v) for doc in sentences])



In [ ]:
topics, first_result = topicmodel(word_vectors, vectorizer, 35, 20, gwa_df['ex_text'], doc_embeddings)

In [ ]:
score(first_result, gwa_df['ex_token'])

0.802636801313551

## _sklearn

In [ ]:

pipe = make_pipeline(
    TfidfVectorizer(),
    TruncatedSVD(100)
)

In [ ]:
class SklearnEmbedder(BaseEmbedder):
    def __init__(self, pipe):
        super().__init__()
        self.pipe = pipe

    def embed(self, documents, verbose=False):
        try:
            check_is_fitted(self.pipe)
            embeddings = self.pipe.transform(documents)
        except NotFittedError:
            embeddings = self.pipe.fit_transform(documents)

        return embeddings

In [ ]:
sklearn_embedder = SklearnEmbedder(pipe)


### 불용어 사전 x

In [ ]:
topics, first_result = topicmodel(sklearn_embedder, vectorizer, 35, 20, gwa_df['text'])

In [ ]:
score(first_result, gwa_df['token'])

0.7073256465440682

### 불용어 사전 o

In [ ]:
topics, first_result = topicmodel(sklearn_embedder, vectorizer, 35, 20, gwa_df['ex_text'])
score(first_result, gwa_df['ex_token'])

0.8048639798954497

## SEBRT

In [ ]:
topics, first_result = topicmodel(model, vectorizer, 35, 20, gwa_df['text'])

In [ ]:
print(score(first_result, gwa_df['token']))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [ ]:
topics, first_result = topicmodel(model, vectorizer, 35, 20, gwa_df['ex_text'])

In [ ]:
print(score(first_result, gwa_df['ex_token']))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

# 최종 최적화

가장 스코어가 높았던 NMF 방법과 불용어 사전을 이용해 토픽모델링을 수행한다

In [ ]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel

# 최적화할 함수 정의
def objective(params):
    # 1. TF-IDF 벡터화
    tfidf_vectorizer = TfidfVectorizer(max_df=params['max_df'], min_df=int(params['min_df']))
    tfidf = tfidf_vectorizer.fit_transform([" ".join(text) for text in texts])

    # 2. NMF 모델링
    nmf_model = NMF(n_components=35, init=params['init'], solver=params['solver'],l1_ratio=params['l1_ratio'], random_state=4).fit(tfidf)

    nmf_topics = nmf_model.transform(tfidf)

    topics_words_list = []
    feature_names = tfidf_vectorizer.get_feature_names_out()
    for topic in nmf_model.components_:
        topic_words = [feature_names[i] for i in topic.argsort()[:-20-1:-1]]
        topics_words_list.append(topic_words)

    dictionary = Dictionary(texts)
    coherence_model = CoherenceModel(topics=topics_words_list, texts=texts, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()

    # 최대화 하려는 경우, score를 음수로 반환
    return {'loss': -coherence_score, 'status': STATUS_OK}

# 파라미터 공간 정의 (n_components 추가)
space = {
    'max_df': hp.uniform('max_df', 0.7, 1.0),
    'min_df': hp.quniform('min_df', 2, 5, 1),
    'l1_ratio': hp.uniform('l1_ratio', 0, 1),
    'init': hp.choice('init', ['random', 'nndsvd', 'nndsvda', 'nndsvdar']),
    'solver': hp.choice('solver', ['cd', 'mu'])
}



In [ ]:
# 최적화 실행
texts = gwa_df['ex_token'].tolist()
trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=100, trials=trials)

print("Best hyperparameters:", best)

100%|██████████| 100/100 [15:10<00:00,  9.11s/trial, best loss: -0.8368117229971074]
Best hyperparameters: {'init': 2, 'l1_ratio': 0.9675387548037933, 'max_df': 0.7322978491623063, 'min_df': 3.0, 'solver': 0}


최종 스코어는 0.8368117229971074 이다.

In [ ]:
# 최적의 파라미터를 사용하여 nmf_topic_modeling 호출
def nmf_topic_modeling(texts, best_params):
    # 1. TF-IDF 벡터화
    tfidf_vectorizer = TfidfVectorizer(max_df=best_params['max_df'], min_df=int(best_params['min_df']))
    tfidf = tfidf_vectorizer.fit_transform([" ".join(text) for text in texts])

    # 2. NMF 모델링
    nmf_model = NMF(
        n_components=35,
        init='nndsvdar',
        solver='cd',
        l1_ratio=best_params['l1_ratio'],
        random_state=4
    ).fit(tfidf)

    # 3. 각 문서에 대해 가장 높은 확률을 가진 토픽 선택
    nmf_topics = nmf_model.transform(tfidf)
    most_prob_topic = nmf_topics.argmax(axis=1)

    # 4. 선택된 토픽과 그에 해당하는 단어들을 데이터 프레임에 저장
    topics_list = []
    topics_words_list = []  # CoherenceModel에 사용할 토픽의 단어 리스트
    feature_names = tfidf_vectorizer.get_feature_names_out()
    for topic_idx, topic in enumerate(nmf_model.components_):
        topic_words = [feature_names[i] for i in topic.argsort()[:-20-1:-1]]  # 상위 20개 단어
        topics_list.append({'Topic': topic_idx, 'Words': " ".join(topic_words)})
        topics_words_list.append(topic_words) # 각 토픽의 단어들을 리스트에 추가

    topics_df = pd.DataFrame(topics_list)

    # 5. Coherence score 계산
    dictionary = Dictionary(texts)
    coherence_model = CoherenceModel(topics=topics_words_list, texts=texts, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()

    return most_prob_topic, topics_df, coherence_score

In [ ]:
# 최적의 파라미터로 함수 호출
topics, topics_df, coherence = nmf_topic_modeling(texts, best)

In [ ]:
coherence

0.8368117229971074

In [ ]:
topics_df
# 확인 결과 잘 나눠진 모습을 확인할 수 있다.

,Topic,Words
0,0,건축 건축물 인허가 안전 공사 공사장 점검 기계 감독 설계 분야 기술 해체 시설물 ...
1,1,급여 의료 자활 보장 기초 수급자 차상위 대상자 결정 연금 주거 생계 생활 복지 변...
2,2,소득세 지방 면허 주민세 징수 부과 자동차세 취득세 사업소 등록 수시 개인 종업원 ...
3,3,건강 금연 방문 예방 보건 접종 의료 정신 진료 코로나 검진 증진 영양 감염병 간호...
4,4,아동 청소년 학대 보호 드림 스타트 친화 권리 복지 여성 돌봄 연계 피해 서비스 양...
5,5,자동차 교통 자전거 등록 화물 이전 방치 유발 운송 교통안전 버스 보험 운행 과태료...
6,6,재정 구정 구청장 편성 지방 심사 자치 성과 규제 혁신 회의 법규 간주 지속 교부금...
7,7,도로 굴착 가로등 제설 공사 복구 보도 조명 시설물 보안등 보수 순찰 감독 포장 설...
8,8,지구 도시 단위 개발 정비 촉진 역세권 디자인 재생 일대 건축 복합 행위 중심 타운...
9,9,어린이집 보육 국공립 여성 가정 육아 부모 가족 교직원 교사 민간 보육료 안심 양성...


In [ ]:
# 토픽값에 따른 부서가 묶인 것을 확인
gwa_df['topic'] = topics

In [ ]:
grouped_values = gwa_df.groupby('topic')['과'].apply(list)

In [ ]:
# 토픽별로 비슷한 과들이 매핑되어 있는 것을 확인할 수 있다.
grouped_values

topic
0     [건축과, 건축과, 건축과, 건축과, 건축과, 건축과, 건축과, 건축지원과, 건축과...
1     [사회복지장애인과, 사회보장과, 생활보장과, 생활보장과, 생활보장과, 생활복지과, ...
2     [세무2과, 지방소득세과, 지방소득세과, 세무2과, 세무2과, 지방소득세과, 세무과...
3     [보건정책과, 보건의료과, 건강관리과, 보건지소, 보건행정과, 질병관리과, 건강관리...
4     [아동청소년과, 가족정책과, 아동청소년과, 청소년과, 아동청소년과, 아동청소년과, ...
5     [교통행정과, 교통행정과, 자동차민원과, 교통행정과, 교통행정과, 교통행정과, 교통...
6     [기획예산과, 기획예산과, 기획예산과, 홍보과, 기획예산과, 기획예산과, 기획예산과...
7     [도로과, 도로관리과, 도로과, 도로관리과, 도로과, 도로관리과, 도로과, 도로관리...
8     [도시계획과, 도시계획과, 도시계획과, 주거정비과, 도시계획과, 도시계획과, 주거정...
9     [가정복지과, 보육지원과, 보육지원과, 가족정책과, 여성가족과, 가족정책과, 여성가...
10    [보건위생과, 위생과, 보건위생과, 보건위생과, 위생관리과, 위생과, 보건위생과, ...
11    [청소과, 자원순환과, 청소행정과, 청소행정과, 자원순환과, 청소행정과, 청소행정과...
12    [체육진흥과, 생활체육과, 생활체육과, 체육진흥과, 체육정책과, 체육진흥과, 생활체...
13    [부동산정보과, 부동산정보과, 부동산정보과, 부동산정보과, 부동산정보과, 부동산정보...
14    [민원여권과, 민원여권과, 민원여권과, 민원여권과, 민원여권과, 민원여권과, 민원여...
15    [일자리청년과, 일자리정책과, 일자리정책과, 일자리지원과, 일자리정책과, 청년정책과...
16    [공원녹지과, 공원녹지과, 푸른도시과, 공원녹지과, 공원녹지과, 공원녹지과, 공원녹...
17    [교통지도과, 주차관리과, 주차행정과, 주차관리과, 주차관리과, 